In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('input.txt', header = None, sep = ',| ', engine = 'python', names = ['x1', 'y1', 'direction', 'x2', 'y2'])
df = df.drop(['direction'], axis = 1)
df

,x1,y1,x2,y2
0,424,924,206,706
1,467,565,432,565
2,722,827,794,899
3,256,172,810,172
4,160,853,148,853
...,...,...,...,...
495,120,191,893,964
496,18,37,969,988
497,134,976,134,689
498,187,842,187,235


In [3]:
# straight line logic
conditions = [df['x1'] == df['x2'], df['y1'] == df['y2']]
choices = ['vertical', 'horizontal']

df['orientation'] = np.select(conditions, choices, default = 'diagonal')
df['straight_flag'] = np.where(df['orientation'] == 'diagonal', 0, 1)


In [4]:
def get_affected_coords(x1, y1, x2, y2):
    '''
    Assumes straight line
    Returns an array of tuple coords
    Tuple required as it's hashable and therefore can be used downstream in the pandas groupby
    
    '''

    coords = []
    # infer direction
    if x1 == x2: # vertical
        small = min(y1, y2)
        big = max(y1, y2)
        x_repeated = np.repeat(x1, big - small + 1)
        y_coords = np.arange(small, big + 1)
        coords = np.stack((x_repeated, y_coords), axis=-1)
    elif y1 == y2: # horizontal
        small = min(x1, x2)
        big = max(x1, x2)
        y_repeated = np.repeat(y1, big - small + 1)
        x_coords = np.arange(small, big + 1)
        coords = np.stack((x_coords, y_repeated), axis = -1)
    else: # diagonal
        small_x = min(x1, x2)
        big_x = max(x1, x2)
        if small_x == x1:
            small_y = y1
            big_y = y2
        else:
            small_y = y2
            big_y = y1
        # we need to go from (small_x, small_y) "up" and "right" one unit
        # through to (big_x, big_y)
        # but we need to account for if the array isn't always positive i.e. left to right
        # so let's use the step parameter of np.arange
        if small_y > big_y:
            step = -1 
        else:
            step = 1
        x_coords = np.arange(small_x, big_x + 1, 1)
        y_coords = np.arange(small_y, big_y + (1 * step), step)
        coords = np.stack((x_coords, y_coords), axis = -1)

    coords_tuple = [tuple(x) for x in coords]
    
    return coords_tuple

In [5]:
df

,x1,y1,x2,y2,orientation,straight_flag
0,424,924,206,706,diagonal,0
1,467,565,432,565,horizontal,1
2,722,827,794,899,diagonal,0
3,256,172,810,172,horizontal,1
4,160,853,148,853,horizontal,1
...,...,...,...,...,...,...
495,120,191,893,964,diagonal,0
496,18,37,969,988,diagonal,0
497,134,976,134,689,vertical,1
498,187,842,187,235,vertical,1


In [6]:
df['affected_coords'] = df.apply(lambda x: get_affected_coords(x['x1'], x['y1'], x['x2'], x['y2']), axis=1)

In [7]:
df

,x1,y1,x2,y2,orientation,straight_flag,affected_coords
0,424,924,206,706,diagonal,0,"[(206, 706), (207, 707), (208, 708), (209, 709..."
1,467,565,432,565,horizontal,1,"[(432, 565), (433, 565), (434, 565), (435, 565..."
2,722,827,794,899,diagonal,0,"[(722, 827), (723, 828), (724, 829), (725, 830..."
3,256,172,810,172,horizontal,1,"[(256, 172), (257, 172), (258, 172), (259, 172..."
4,160,853,148,853,horizontal,1,"[(148, 853), (149, 853), (150, 853), (151, 853..."
...,...,...,...,...,...,...,...
495,120,191,893,964,diagonal,0,"[(120, 191), (121, 192), (122, 193), (123, 194..."
496,18,37,969,988,diagonal,0,"[(18, 37), (19, 38), (20, 39), (21, 40), (22, ..."
497,134,976,134,689,vertical,1,"[(134, 689), (134, 690), (134, 691), (134, 692..."
498,187,842,187,235,vertical,1,"[(187, 235), (187, 236), (187, 237), (187, 238..."


In [8]:
all_affected_coords = df.explode(column = 'affected_coords')

In [9]:
def total_affected_coords(exclude_diagonal = True):
    if exclude_diagonal == True:
        df = all_affected_coords[all_affected_coords['straight_flag'] == 1]
    else:
        df = all_affected_coords
    df_grouped = df[['affected_coords', 'orientation']].groupby('affected_coords').count()
    df_grouped.columns = ['number_of_intersecting_lines']
    return df_grouped

In [10]:
# affected coordinates for part one
df_part_one = total_affected_coords(exclude_diagonal = True)
answer_one = len(df_part_one[df_part_one['number_of_intersecting_lines'] > 1])
answer_one

5147

In [11]:
# affected coordinates for part two
df_part_two = total_affected_coords(exclude_diagonal = False)
answer_two = len(df_part_two[df_part_two['number_of_intersecting_lines'] > 1])
answer_two

16925